# Feedbacks CERES via SW fluxes

(1) Reconstruit les données avec les PCs et EOFs des flux SW. 

(2) Multi Linéaire Régression des PCs sur les feedbacks pour trouver les coefficients Beta. 

(3) Retrouver les PCs des observations CERES. 

(4) Prédire les feedbacks des observations.

# Importe modules

In [1]:
# Computational modules 
%matplotlib inline
import xarray as xr
import glob
import os
import numpy as np
import netCDF4
from netCDF4 import Dataset
import pandas as pd
import re
from array import array
from pylab import *
#import geopandas
from eofs.xarray import Eof
from eofs.multivariate.standard import MultivariateEof

# Plotting modules 
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import pandas.plotting
import matplotlib.ticker as ticker
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import BoundaryNorm

# Scikit-learn
from sklearn import linear_model
from sklearn.linear_model import LassoCV, LassoLarsCV, LassoLarsIC
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.neural_network import MLPRegressor

# Functions definition

In [2]:
import sys
sys.path.append('/data/home/globc/peatier/CNRMppe')
import Fonctions
from Fonctions import get_wavg_budget_df
from Fonctions import wavg 
from Fonctions import plotlines_Xdf
from Fonctions import plotlines_1df
from Fonctions import Deltas_Lambda
from Fonctions import get_3D_budget_xarr
from Fonctions import get_3D_xarr
from Fonctions import get_3D_SW_xarr
from Fonctions import get_3D_LW_xarr

# EOF, pr_mean and PCs

In [3]:
Mean =  pd.read_pickle('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_pr_mean.npy').to_xarray().to_array()
Mean = Mean[0,:,:].rename({'variable':'mode'})
Mean

<xarray.DataArray (lat: 128, lon: 256)>
array([[0.16115941, 0.16115941, 0.16115941, ..., 0.16115941, 0.16115941,
        0.16115941],
       [0.08960055, 0.08960055, 0.08960055, ..., 0.08960055, 0.08960055,
        0.08960055],
       [0.14128352, 0.14128352, 0.14128352, ..., 0.15915012, 0.14128352,
        0.14128352],
       ...,
       [0.70739219, 0.70739219, 0.70739219, ..., 0.64911432, 0.70739219,
        0.70739219],
       [0.64713049, 0.64713049, 0.64713049, ..., 0.64713049, 0.64713049,
        0.64713049],
       [0.55278562, 0.55278562, 0.55278562, ..., 0.55278562, 0.55278562,
        0.55278562]])
Coordinates:
  * lat      (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
  * lon      (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
    mode     <U6 'prmean'

In [4]:
Anom =  pd.read_pickle('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_pr_anom.npy').to_xarray().to_array()
Anom = Anom[0,:,:].rename({'variable':'mode'})
Anom

<xarray.DataArray (time: 103, lat: 128, lon: 256)>
array([[[ 0.04604292,  0.04604292,  0.04604292, ...,  0.04604292,
          0.04604292,  0.04604292],
        [ 0.00349467,  0.00349467,  0.00349467, ...,  0.00349467,
          0.00349467,  0.00349467],
        [-0.00546162, -0.00546162, -0.00546162, ..., -0.00426472,
         -0.00546162, -0.00546162],
        ...,
        [ 0.0898523 ,  0.0898523 ,  0.0898523 , ...,  0.07462792,
          0.0898523 ,  0.0898523 ],
        [ 0.03283331,  0.03283331,  0.03283331, ...,  0.03283331,
          0.03283331,  0.03283331],
        [ 0.0174318 ,  0.0174318 ,  0.0174318 , ...,  0.0174318 ,
          0.0174318 ,  0.0174318 ]],

       [[-0.01311719, -0.01311719, -0.01311719, ..., -0.01311719,
         -0.01311719, -0.01311719],
        [-0.01303413, -0.01303413, -0.01303413, ..., -0.01303413,
         -0.01303413, -0.01303413],
        [-0.02087199, -0.02087199, -0.02087199, ..., -0.01617833,
         -0.02087199, -0.02087199],
        ...,
        [ 0.150107  ,  0.150107  ,  0.150107  , ...,  0.13492018,
          0.150107  ,  0.150107  ],
        [ 0.11547123,  0.11547123,  0.11547123, ...,  0.11547123,
          0.11547123,  0.11547123],
        [ 0.02428303,  0.02428303,  0.02428303, ...,  0.02428303,
          0.02428303,  0.02428303]],

       [[-0.02046119, -0.02046119, -0.02046119, ..., -0.02046119,
         -0.02046119, -0.02046119],
        [-0.00103315, -0.00103315, -0.00103315, ..., -0.00103315,
         -0.00103315, -0.00103315],
        [ 0.00298186,  0.00298186,  0.00298186, ...,  0.00034031,
          0.00298186,  0.00298186],
        ...,
        [-0.0638615 , -0.0638615 , -0.0638615 , ..., -0.0539646 ,
         -0.0638615 , -0.0638615 ],
        [-0.05843218, -0.05843218, -0.05843218, ..., -0.05843218,
         -0.05843218, -0.05843218],
        [-0.0170032 , -0.0170032 , -0.0170032 , ..., -0.0170032 ,
         -0.0170032 , -0.0170032 ]],

       ...,

       [[ 0.01738025,  0.01738025,  0.01738025, ...,  0.01738025,
          0.01738025,  0.01738025],
        [ 0.01857094,  0.01857094,  0.01857094, ...,  0.01857094,
          0.01857094,  0.01857094],
        [ 0.02014906,  0.02014906,  0.02014906, ...,  0.02145709,
          0.02014906,  0.02014906],
        ...,
        [-0.0559437 , -0.0559437 , -0.0559437 , ..., -0.05444218,
         -0.0559437 , -0.0559437 ],
        [-0.02873785, -0.02873785, -0.02873785, ..., -0.02873785,
         -0.02873785, -0.02873785],
        [ 0.04868053,  0.04868053,  0.04868053, ...,  0.04868053,
          0.04868053,  0.04868053]],

       [[-0.00432976, -0.00432976, -0.00432976, ..., -0.00432976,
         -0.00432976, -0.00432976],
        [-0.00572422, -0.00572422, -0.00572422, ..., -0.00572422,
         -0.00572422, -0.00572422],
        [-0.02991085, -0.02991085, -0.02991085, ..., -0.035116  ,
         -0.02991085, -0.02991085],
        ...,
        [-0.06772801, -0.06772801, -0.06772801, ..., -0.06410834,
         -0.06772801, -0.06772801],
        [-0.0573616 , -0.0573616 , -0.0573616 , ..., -0.0573616 ,
         -0.0573616 , -0.0573616 ],
        [-0.025025  , -0.025025  , -0.025025  , ..., -0.025025  ,
         -0.025025  , -0.025025  ]],

       [[ 0.01536465,  0.01536465,  0.01536465, ...,  0.01536465,
          0.01536465,  0.01536465],
        [ 0.01310458,  0.01310458,  0.01310458, ...,  0.01310458,
          0.01310458,  0.01310458],
        [-0.0143301 , -0.0143301 , -0.0143301 , ..., -0.01517536,
         -0.0143301 , -0.0143301 ],
        ...,
        [-0.12916062, -0.12916062, -0.12916062, ..., -0.12302981,
         -0.12916062, -0.12916062],
        [-0.12143837, -0.12143837, -0.12143837, ..., -0.12143837,
         -0.12143837, -0.12143837],
        [-0.0792483 , -0.0792483 , -0.0792483 , ..., -0.0792483 ,
         -0.0792483 , -0.0792483 ]]])
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9 ... 94 95 96 97 98 99 100 101 102
  * lat      (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
  * 

In [5]:
eof1 = pd.read_pickle('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOFmap1_pr.npy').to_xarray().to_array()
eof2 = pd.read_pickle('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOFmap2_pr.npy').to_xarray().to_array()
eof3 = pd.read_pickle('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOFmap3_pr.npy').to_xarray().to_array()
eof4 = pd.read_pickle('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOFmap4_pr.npy').to_xarray().to_array()
eof5 = pd.read_pickle('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOFmap5_pr.npy').to_xarray().to_array()

eof1 = eof1[0,:,:].rename({'variable':'mode'})
eof2 = eof2[0,:,:].rename({'variable':'mode'})
eof3 = eof3[0,:,:].rename({'variable':'mode'})
eof4 = eof4[0,:,:].rename({'variable':'mode'})
eof5 = eof5[0,:,:].rename({'variable':'mode'})

eofs = [eof1,eof2,eof3,eof4,eof5]

In [6]:
eofs_combined = xr.combine_nested(eofs, concat_dim='eofs')
#eofs_combined

In [7]:
pc1 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOF1pc_pr.npy')
pc2 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOF2pc_pr.npy')
pc3 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOF3pc_pr.npy')
pc4 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOF4pc_pr.npy')
pc5 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/PPE2_EOF5pc_pr.npy')

PC_df = pd.DataFrame(pc1)
PC_df['pc1'] = pc1
PC_df['pc2'] = pc2
PC_df['pc3'] = pc3
PC_df['pc4'] = pc4
PC_df['pc5'] = pc5
PC_df = PC_df.drop(labels = 0, axis=1)
#PC_df

PC_df

,pc1,pc2,pc3,pc4,pc5
0,-0.576649,-1.244866,0.704543,-0.674594,0.819153
1,-0.862754,0.330346,0.536533,-0.293066,-0.119754
2,-0.332272,0.158484,0.630465,-0.688250,1.728777
3,-1.847503,-0.166455,0.771956,-0.095791,-0.242596
4,1.070163,-0.001736,1.908538,1.561213,2.090414
...,...,...,...,...,...
97,-1.411826,0.396202,0.066118,0.040638,0.571056
98,0.870097,1.022319,-0.785519,-0.803624,-0.080335
99,0.344547,0.194929,0.032738,-0.573655,-0.236374
100,0.470452,0.240466,1.254922,-0.334611,-0.204161


In [10]:
#pc1_official = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/official_EOF1pc_pr.npy')
#pc2_official = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/official_EOF2pc_pr.npy')
#pc3_official = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/official_EOF3pc_pr.npy')
#pc4_official = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/official_EOF4pc_pr.npy')
#pc5_official = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/official_EOF5pc_pr.npy')

pc1_p1 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/p1_EOF1pc_pr.npy')
pc2_p1 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/p1_EOF2pc_pr.npy')
pc3_p1 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/p1_EOF3pc_pr.npy')
pc4_p1 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/p1_EOF4pc_pr.npy')
pc5_p1 = np.load('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/p1_EOF5pc_pr.npy')

In [11]:
PCs = [0]*102
for i in range(0,102,1):
    tmp = PC_df.iloc[i].values
    PCs[i] = tmp

#PCs

[array([-0.57664938, -1.24486621,  0.70454319, -0.67459379,  0.8191526 ]),
 array([-0.862754  ,  0.33034635,  0.53653296, -0.29306621, -0.11975398]),
 array([-0.33227233,  0.15848448,  0.6304651 , -0.68824993,  1.72877654]),
 array([-1.8475028 , -0.16645463,  0.77195588, -0.09579112, -0.24259649]),
 array([ 1.07016323e+00, -1.73643337e-03,  1.90853757e+00,  1.56121296e+00,
         2.09041412e+00]),
 array([ 0.33726264,  0.50359117, -0.12102003, -0.65171432,  0.5337709 ]),
 array([-0.6317747 , -2.41628299,  0.21845827, -0.40451857,  0.64569079]),
 array([ 0.16583141,  0.01315336, -1.21382996, -0.77484445,  0.25954776]),
 array([ 0.86624383, -1.37954222, -1.01719021,  0.84249381, -1.33378542]),
 array([-0.99284765,  0.37804902, -1.13503048, -0.59673134,  0.79226104]),
 array([ 0.02543915, -0.63170472, -1.27624312,  0.15559035,  1.45833621]),
 array([-1.67251061, -0.57406539, -0.26380027,  1.57264153, -1.05338275]),
 array([-0.71969156,  1.6846796 , -0.18945488, -0.01577365, -0.76074555]

# Trouver U_obs : Reconstruciton des données

X_ens = U_ens(pc) * v_ens(eof) 

Compute anomaly : X_obs = X_obs - SW_mean_ens 

X_obs = U_obs * v_ens 

U_obs = X_obs * inv(v_ens)

## Get the CMIP6 datatset

In [22]:
def load_monthly_clim(path, filename, variables) :
    
    file = xr.open_mfdataset(path+filename,combine='by_coords')
    df=file[variables].to_dataframe()
    
    # Compute Climatological Annual Cycle :
    df1=df.reset_index(level=['time', 'lat', 'lon'])
    df1['month']=pd.DatetimeIndex(df1['time']).month # time in years
    df_mean=df1.groupby(['month', 'lat', 'lon']).mean()
    
    return df_mean

In [23]:
# Download the amip dataframes and compute the radiative budget  

path = "/data/scratch/globc/peatier/CMIP6/CNRM-CM6-1/CFMIP/amip/"
filename = "*_amip_*.nc"
variables = ['pr']
CNRM_amip = load_monthly_clim(path, filename, variables)

path = "/data/scratch/globc/peatier/CMIP6/MRI-ESM2-0/amip/"
MRI_amip = load_monthly_clim(path, filename, variables)

path = "/data/scratch/globc/peatier/CMIP6/CanESM5/amip/"
filename = "standard_*_amip_*regrid.nc"
CAN_amip = load_monthly_clim(path, filename, variables)
CAN_amip

path = "/data/scratch/globc/peatier/CMIP6/MIROC6/"
filename = "*_amip_*regrid.nc"
MIROC_amip = load_monthly_clim(path, filename, variables)
MIROC_amip

path = "/data/scratch/globc/peatier/CMIP6/HadGEM3-GC31-LL/"
filename = "standard_*_amip_*regrid.nc"
HadGEM3_amip = load_monthly_clim(path, filename, variables)
HadGEM3_amip

path = "/data/scratch/globc/peatier/CMIP6/IPSL-CM6A-LR/"
filename = "*_amip_*regrid.nc"
IPSL_amip = load_monthly_clim(path, filename, variables)
IPSL_amip

path = "/data/scratch/globc/peatier/CMIP6/BCC-CSM2-MR/"
filename = "standard_*_amip_*regrid.nc"
BCC_amip = load_monthly_clim(path, filename, variables)
BCC_amip

pr  height
month lat        lon                        
1     -88.927735 0.00000    0.000002     2.0
                 1.40625    0.000002     2.0
                 2.81250    0.000002     2.0
                 4.21875    0.000002     2.0
                 5.62500    0.000002     2.0
...                              ...     ...
12     88.927735 352.96875  0.000006     2.0
                 354.37500  0.000006     2.0
                 355.78125  0.000006     2.0
                 357.18750  0.000006     2.0
                 358.59375  0.000006     2.0

[393216 rows x 2 columns]

In [24]:
df_obs = pd.DataFrame()
df_obs['CNRM'] = CNRM_amip[variables[0]]
df_obs['MRI'] = MRI_amip[variables[0]]
df_obs['CAN'] = CAN_amip[variables[0]]
df_obs['MIROC'] = MIROC_amip[variables[0]]
df_obs['HadGEM3'] = HadGEM3_amip[variables[0]]
df_obs['IPSL'] = IPSL_amip[variables[0]]
df_obs['BCC'] = BCC_amip[variables[0]]
#df_obs = df_obs.drop(['rsut', 'height'], axis = 1)
df_obs

CNRM       MRI       CAN     MIROC   HadGEM3  \
month lat        lon                                                           
1     -88.927735 0.00000    0.000002  0.000002  0.000002  0.000002  0.000002   
                 1.40625    0.000002  0.000002  0.000002  0.000002  0.000002   
                 2.81250    0.000002  0.000002  0.000002  0.000002  0.000002   
                 4.21875    0.000002  0.000002  0.000002  0.000002  0.000002   
                 5.62500    0.000002  0.000002  0.000002  0.000002  0.000002   
...                              ...       ...       ...       ...       ...   
12     88.927735 352.96875  0.000006  0.000005  0.000006  0.000003  0.000005   
                 354.37500  0.000006  0.000005  0.000006  0.000003  0.000005   
                 355.78125  0.000006  0.000005  0.000006  0.000003  0.000005   
                 357.18750  0.000006  0.000005  0.000006  0.000003  0.000005   
                 358.59375  0.000006  0.000005  0.000006  0.000003  0.000005   

                                IPSL       BCC  
month lat        lon                            
1     -88.927735 0.00000    0.000001  0.000002  
                 1.40625    0.000001  0.000002  
                 2.81250    0.000001  0.000002  
                 4.21875    0.000001  0.000002  
                 5.62500    0.000001  0.000002  
...                              ...       ...  
12     88.927735 352.96875  0.000006  0.000006  
                 354.37500  0.000006  0.000006  
                 355.78125  0.000006  0.000006  
                 357.18750  0.000006  0.000006  
                 358.59375  0.000006  0.000006  

[393216 rows x 7 columns]

In [25]:
obs = df_obs.groupby(['lat','lon']).mean().to_xarray()*86400 
obs

<xarray.Dataset>
Dimensions:  (lat: 128, lon: 256)
Coordinates:
  * lon      (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
  * lat      (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
Data variables:
    CNRM     (lat, lon) float64 0.1992 0.1992 0.1992 ... 0.6535 0.6535 0.6535
    MRI      (lat, lon) float64 0.1527 0.152 0.1513 ... 0.7101 0.7103 0.7106
    CAN      (lat, lon) float64 0.1428 0.1428 0.1398 ... 0.6491 0.6418 0.6418
    MIROC    (lat, lon) float64 0.104 0.1035 0.1031 ... 0.4774 0.4776 0.4779
    HadGEM3  (lat, lon) float64 0.1668 0.1664 0.1661 ... 0.6522 0.6528 0.6535
    IPSL     (lat, lon) float64 0.1578 0.1551 0.1527 ... 0.6418 0.647 0.6516
    BCC      (lat, lon) float64 0.198 0.1968 0.1956 ... 0.6695 0.6703 0.67

## Compute Anomaly

In [26]:
X_obs = {}
X_obs['CNRM'] = obs['CNRM'] - Mean
X_obs['MRI'] = obs['MRI'] - Mean
X_obs['CAN'] = obs['CAN'] - Mean
X_obs['MIROC'] = obs['MIROC'] - Mean
X_obs['HadGEM3'] = obs['HadGEM3'] - Mean
X_obs['IPSL'] = obs['IPSL'] - Mean
X_obs['BCC'] = obs['BCC'] - Mean
X_obs

{'CNRM': <xarray.DataArray (lat: 128, lon: 256)>
 array([[ 0.03803879,  0.03803879,  0.03803879, ...,  0.03803879,
          0.03803879,  0.03803879],
        [-0.00180562, -0.00180562, -0.00180562, ..., -0.00180562,
         -0.00180562, -0.00180562],
        [ 0.00881006,  0.00881006,  0.00881006, ...,  0.01136503,
          0.00881006,  0.00881006],
        ...,
        [ 0.14970176,  0.14970176,  0.14970176, ...,  0.13433107,
          0.14970176,  0.14970176],
        [ 0.1349773 ,  0.1349773 ,  0.1349773 , ...,  0.1349773 ,
          0.1349773 ,  0.1349773 ],
        [ 0.10075392,  0.10075392,  0.10075392, ...,  0.10075392,
          0.10075392,  0.10075392]])
 Coordinates:
   * lon      (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
   * lat      (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
     mode     <U6 'prmean', 'MRI': <xarray.DataArray (lat: 128, lon: 256)>
 array([[-0.00846083, -0.0091534 , -0.00983765, ..., -0.00624798,
         -0.00

## Trouver U_obs

In [27]:
eofs_inv = np.linalg.pinv(eofs_combined.transpose())
#eofs_inv = xr.DataArray(eofs_inv, dims=['eofs','lon','lat'])
eofs_inv

array([[[-8.45191231e-03, -8.46400724e-03, -2.95886726e-02, ...,
         -2.90068827e-02, -2.62095083e-02, -1.39444433e-02],
        [ 1.13883098e-02,  1.09127532e-02,  2.42993446e-02, ...,
          2.88350590e-02,  2.64480770e-03, -2.90236342e-03],
        [-5.62291328e-02, -3.48746531e-02, -9.45414779e-02, ...,
         -1.35316954e-01, -8.66398218e-02, -6.96893398e-03],
        [-8.31623910e-03,  1.30134156e-02,  3.97802838e-02, ...,
          1.12110060e-02, -1.47637423e-02,  3.79287915e-02],
        [ 5.70449088e-02,  1.77652259e-02,  2.62363219e-02, ...,
          1.07748989e-01,  5.32129109e-02, -6.19922052e-02]],

       [[-7.18569213e-03, -6.25070766e-03, -2.15050909e-02, ...,
         -2.50829037e-02, -2.10663917e-02, -9.97513767e-03],
        [ 6.59373245e-03,  6.08695881e-03,  1.33588306e-02, ...,
          1.85592432e-02,  2.22548665e-03, -2.88097100e-03],
        [-5.65386626e-02, -2.93758038e-02, -7.88598253e-02, ...,
         -1.34379401e-01, -9.07492890e-02, -4.71523

In [28]:
X_CNRM_bis = X_obs['CNRM'].expand_dims(dim={'dummy':range(0,102,1)})
X_MRI_bis = X_obs['MRI'].expand_dims(dim={'dummy':range(0,102,1)})
X_CAN_bis = X_obs['CAN'].expand_dims(dim={'dummy':range(0,102,1)})
X_MIROC_bis = X_obs['MIROC'].expand_dims(dim={'dummy':range(0,102,1)})
X_HadGEM3_bis = X_obs['HadGEM3'].expand_dims(dim={'dummy':range(0,102,1)})
X_IPSL_bis = X_obs['IPSL'].expand_dims(dim={'dummy':range(0,102,1)})
X_BCC_bis = X_obs['BCC'].expand_dims(dim={'dummy':range(0,102,1)})

In [29]:
U_CNRM = np.dot(eofs_inv,X_CNRM_bis.transpose('lon','lat', 'dummy'))
U_MRI = np.dot(eofs_inv,X_MRI_bis.transpose('lon','lat', 'dummy'))
U_CAN = np.dot(eofs_inv,X_CAN_bis.transpose('lon','lat', 'dummy'))
U_MIROC = np.dot(eofs_inv,X_MIROC_bis.transpose('lon','lat', 'dummy'))
U_HadGEM3 = np.dot(eofs_inv,X_HadGEM3_bis.transpose('lon','lat', 'dummy'))
U_IPSL = np.dot(eofs_inv,X_IPSL_bis.transpose('lon','lat', 'dummy'))
U_BCC = np.dot(eofs_inv,X_BCC_bis.transpose('lon','lat', 'dummy'))


In [30]:
U_CNRM = xr.DataArray(U_CNRM, dims=['lon','eofs','lon2', 'dummy'])
U_CNRM_avg = U_CNRM[0,:,0,0].transpose()

U_MRI = xr.DataArray(U_MRI, dims=['lon','eofs','lon2', 'dummy'])
U_MRI_avg = U_MRI[0,:,0,0].transpose()

U_CAN = xr.DataArray(U_CAN, dims=['lon','eofs','lon2', 'dummy'])
U_CAN_avg = U_CAN[0,:,0,0].transpose()

U_MIROC = xr.DataArray(U_MIROC, dims=['lon','eofs','lon2', 'dummy'])
U_MIROC_avg = U_MIROC[0,:,0,0].transpose()

U_HadGEM3 = xr.DataArray(U_HadGEM3, dims=['lon','eofs','lon2', 'dummy'])
U_HadGEM3_avg = U_HadGEM3[0,:,0,0].transpose()

U_IPSL = xr.DataArray(U_IPSL, dims=['lon','eofs','lon2', 'dummy'])
U_IPSL_avg = U_IPSL[0,:,0,0].transpose()

U_BCC = xr.DataArray(U_BCC, dims=['lon','eofs','lon2', 'dummy'])
U_BCC_avg = U_BCC[0,:,0,0].transpose()

In [31]:
#Save the U_obs_avg 
np.save('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/U_CNRM_pr.npy', U_CNRM_avg)
np.save('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/U_MRI_pr.npy', U_MRI_avg)
np.save('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/U_CAN_pr.npy', U_CAN_avg)
np.save('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/U_MIROC_pr.npy', U_MIROC_avg)
np.save('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/U_HadGEM3_pr.npy', U_HadGEM3_avg)
np.save('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/U_IPSL_pr.npy', U_IPSL_avg)
np.save('/data/home/globc/peatier/CNRMppe/PPE/ENSEMBLE2/files/npy/U_BCC_pr.npy', U_BCC_avg)